# Config

In [1]:
%run Config.py

# Data

In [2]:
!!python ../OpenNMT-py/preprocess.py --train_src "../data/cornell movie-dialogs corpus/src_movie_lines.txt" --train_tgt "../data/cornell movie-dialogs corpus/tgt_movie_lines.txt" --save_data ../data/cornell_raw

['Please backup existing pt files: ../data/cornell_raw.train*.pt, to avoid overwriting them!']

In [3]:
!!python ../OpenNMT-py/preprocess.py --train_src "../data/cornell movie-dialogs corpus/src_movie_lines.txt" --train_tgt "../data/cornell movie-dialogs corpus/tgt_movie_lines.txt" --save_data ../data/cornell_raw_min_30_10 --src_words_min_frequency 30 --tgt_words_min_frequency 30 --src_seq_length 10 --tgt_seq_length 10

['Please backup existing pt files: ../data/cornell_raw_min_30_10.train*.pt, to avoid overwriting them!']

In [4]:
vocab_fields = torch.load("../data/cornell_raw_min_30_10.vocab.pt")

In [5]:
src_text_field = vocab_fields["src"].base_field
src_vocab = src_text_field.vocab
src_padding = src_vocab.stoi[src_text_field.pad_token] #stoi: mapping token strings to numerical identifiers.
# ['<unk>', '<blank>', 'I', 'you', 'the', 'to', 'a', 'of', 'and', 'You']
# src_text_field.pad_token : '<blank>'

tgt_text_field = vocab_fields['tgt'].base_field
tgt_vocab = tgt_text_field.vocab
tgt_padding = tgt_vocab.stoi[tgt_text_field.pad_token]

In [6]:
config.src_vocab_size = len(src_vocab)
config.tgt_vocab_size = len(tgt_vocab)
config.src_padding = src_padding
config.tgt_padding = tgt_padding

In [7]:
config.tgt_unk = tgt_vocab.stoi[tgt_text_field.unk_token]
config.tgt_bos = tgt_vocab.stoi[tgt_text_field.init_token]
config.tgt_eos = tgt_vocab.stoi[tgt_text_field.eos_token]

In [8]:
config.src_vocab = src_vocab
config.tgt_vocab = tgt_vocab

# seq2seq-DQN

In [9]:
%run modules/NoisyLinear.py

In [10]:
%run modules/DQN.py

In [111]:
%run modules/Model.py

In [112]:
model = Model(config, DQN)

In [113]:
model.current_model

DQN(
  (encoder_embeddings): Embeddings(
    (make_embedding): Sequential(
      (emb_luts): Elementwise(
        (0): Embedding(1462, 100, padding_idx=1)
      )
    )
  )
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(1462, 100, padding_idx=1)
        )
      )
    )
    (rnn): LSTM(100, 250, bidirectional=True)
  )
  (decoder_embeddings): Embeddings(
    (make_embedding): Sequential(
      (emb_luts): Elementwise(
        (0): Embedding(1472, 100, padding_idx=1)
      )
    )
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(1472, 100, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.0)
    (rnn): StackedLSTM(
      (dropout): Dropout(p=0.0)
      (layers): ModuleList(
        (0): LSTMCell(600, 500)
      )
    )
    (attn): GlobalAttention(
      (linear

In [114]:
%run modules/MSELoss.py

In [115]:
loss = MSELoss(
    nn.MSELoss(reduction="none"),
    model.current_model.generator
)

In [116]:
%run modules/Reward.py

In [117]:
config.rewards = ['BLEU']
config.rewards_weights = [1]    

reward = Reward(config)

In [118]:
lr = 1
torch_optimizer = torch.optim.SGD(model.current_model.parameters(), lr=lr)
optim = onmt.utils.optimizers.Optimizer(torch_optimizer, learning_rate=lr, max_grad_norm=2)

In [119]:
#report_manager = onmt.utils.ReportMgr(report_every=1, start_time=None, tensorboard_writer=None)

In [150]:
from itertools import chain
train_data_file = "../data/cornell_raw_min_30_10.train.0.pt"
train_iter = onmt.inputters.inputter.DatasetLazyIter(dataset_paths=[train_data_file],
                                                     fields=vocab_fields,
                                                     batch_size=1,
                                                     batch_size_multiple=1,
                                                     batch_size_fn=None,
                                                     device="cpu",
                                                     is_train=True,
                                                     repeat=False,
                                                     pool_factor=8192)

# Preload Experience Replay Buffer

if len(model.replay_memory) == 0:
    data = list(train_iter)
    for example in data[:config.PRELOADING_SIZE]:
        model.replay_memory.preload(example.src[0].squeeze(1), example.tgt.squeeze(1), 1)
        model.sample_buffer.preload(example.src[0].squeeze(1), example.tgt.squeeze(1), None)

In [21]:
%run modules/QLearning.py

In [22]:
trainer = QLearning(config,
                    model,
                    reward=reward,
                    train_loss=loss,
                    valid_loss=loss,
                    optim=optim,
                    gpu_verbose_level=100)
                    #shard_size = 0

In [23]:
#for i in model.target_model.parameters():
#    print(i.abs().sum())

In [24]:
#for i, t in enumerate(model.replay_memory._storage):
#    if t[1].size(0) > 10:
#        print(i, t[1].size(0), t[1])
    
    #if t[1][-1].item() == 96:
    #    print(i, t[1][-1])
    #    print(t[1].size())
    #print((t[1] == 0).sum())
    #print(t[1].size(0))
    
#for x in model.replay_memory._storage:
#    #print([t for t in x[1]])
#    print(' '.join([tgt_vocab.itos[t.item()] for t in x[1]]))

#for i, x in enumerate(data[:config.PRELOADING_SIZE]):
#    print(' '.join([tgt_vocab.itos[token] for token in x.src[0].squeeze().tolist()]))

In [25]:
result = trainer.train(train_steps=400, valid_steps=200)

INFO:root:Start training loop and validate every 200 steps...
INFO:root:Step 1


BETA 0.40006


INFO:root:Batch Length: tensor([10, 10, 10,  9,  9,  9,  9,  9,  8,  7,  6,  6,  6,  6,  6,  6,  6,  5,
         5,  4,  4,  3,  3,  3,  3,  3,  2,  1,  1,  1,  1,  1],
       dtype=torch.int16)
INFO:root:tensor(1.0857, grad_fn=<DivBackward0>)
INFO:root:Step 2


BETA 0.40012000000000003


INFO:root:Batch Length: tensor([10, 10, 10, 10, 10,  9,  9,  9,  9,  9,  8,  8,  8,  7,  7,  7,  7,  6,
         6,  5,  5,  5,  4,  3,  3,  3,  2,  2,  2,  2,  2,  1],
       dtype=torch.int16)
INFO:root:tensor(0.3430, grad_fn=<DivBackward0>)
INFO:root:Step 3


...

INFO:root:Using / Replacing Index 325


I I </s> (0.5372849702835083)


INFO:root:Using / Replacing Index 326


I I I I I I I I I I I I (0.04088987037539482)


KeyboardInterrupt: 